<a href="https://colab.research.google.com/github/iroliak/ekpa_1/blob/main/DarkNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Πρακτική Εφαρμογή: Ανίχνευση Απειλών και Ανωμαλιών στο Σκοτεινό Διαδίκτυο

## Εισαγωγή

Η πρακτική εφαρμογή εστιάζει στους τρόπους με τους οποίους μπορούν τα ευφυή συστήματα ΤΝ και οι αλγόριθμοι μηχανικής μάθησης να χρησιμοποιηθούν για την υλοποίηση μηχανισμών ενεργητικής ασφάλειας που μπορούν να θωρακίσουν τα σύγχρονα πληροφοριακά συστήματα.

## Μεθοδολογία

### Φόρτωση και Προεπεξεργασία Δεδομένων

In [ ]:
import pandas as pd

# Φόρτωση δεδομένων
url = "https://raw.githubusercontent.com/kdemertzis/EKPA/main/Data/DarkNet.csv"
data = pd.read_csv(url)

# Εμφάνιση πρώτων γραμμών του dataset
print(data.head())


          Src_IP  Src_Port          Dst_IP  Dst_Port  Protocol  Flow_Duration  \
0  10.152.152.11     57158   216.58.220.99       443         6            229   
1  10.152.152.11     57159   216.58.220.99       443         6            407   
2  10.152.152.11     57160   216.58.220.99       443         6            431   
3  10.152.152.11     49134  74.125.136.120       443         6            359   
4  10.152.152.11     34697  173.194.65.127     19305         6       10778451   

   Total_Fwd_Packet  Total_Bwd_packets  Total_Length_of_Fwd_Packet  \
0                 1                  1                           0   
1                 1                  1                           0   
2                 1                  1                           0   
3                 1                  1                           0   
4               591                400                       64530   

   Total_Length_of_Bwd_Packet  ...  Active_Mean  Active_Std  Active_Max  \
0                

<ipython-input-3-9a6f0510b17c>:5: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(url)


### Εξερευνητική Ανάλυση Δεδομένων

In [ ]:
# Εντοπισμός ελλιπών τιμών
missing_values = data.isnull().sum()

# Εμφάνιση στηλών με ελλιπείς τιμές
missing_values[missing_values > 0]


Series([], dtype: int64)

### Δημιουργία και Εκπαίδευση Μοντέλων

In [ ]:
import numpy as np

# Έλεγχος για απείρως μεγάλες τιμές
numerical_columns = data.select_dtypes(include=['float64', 'int64']).columns
infinite_values = data[numerical_columns].isin([np.inf, -np.inf]).sum()

# Εμφάνιση στηλών με απείρως μεγάλες τιμές
infinite_values[infinite_values > 0]


Flow_Packets/s    49
dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

# Διαχωρισμός σε σύνολα εκπαίδευσης και δοκιμής
X = data.drop(columns=['Label-1', 'Label-2'])
y = data['Label-1']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Εμφάνιση του διαμορφωμένου dataset
X_train.head(), y_train.head()


(              Src_IP  Src_Port         Dst_IP  Dst_Port  Protocol  \
 41605  10.152.152.11 -1.584368  178.65.230.88  2.495546  1.078884   
 34682     10.8.8.130  0.870768   129.98.43.99  0.484143 -0.893271   
 23701      10.0.2.15  0.479050  74.125.207.16 -0.679192 -0.893271   
 37430  10.152.152.11  0.083921   108.219.11.8  0.746394 -0.893271   
 27423   54.93.145.72 -2.050625  10.152.152.11  1.129396 -0.893271   
 
        Flow_Duration  Total_Fwd_Packet  Total_Bwd_packets  \
 41605      -0.365264         -0.084829          -0.058447   
 34682       1.890850         -0.072479          -0.050857   
 23701      -0.594290         -0.085123          -0.058447   
 37430      -0.580071         -0.084829          -0.057832   
 27423      -0.594292         -0.085417          -0.058242   
 
        Total_Length_of_Fwd_Packet  Total_Length_of_Bwd_Packet  ...  \
 41605                   -0.048994                   -0.037097  ...   
 34682                   -0.048939                   -0.036534

In [15]:
# Αφαίρεση των στηλών Src_IP και Dst_IP
X = data.drop(columns=['Label-1', 'Label-2', 'Src_IP', 'Dst_IP'])

# Διαχωρισμός σε σύνολα εκπαίδευσης και δοκιμής
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Δημιουργία και εκπαίδευση του μοντέλου Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Πρόβλεψη στο σύνολο δοκιμής
y_pred = rf_model.predict(X_test)

# Αξιολόγηση του μοντέλου
confusion_mtx = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

confusion_mtx, class_report


(array([[9227,    4,    0,   12],
        [   4, 6618,    5,  135],
        [   3,   42,  374,    1],
        [   7,  324,    2, 3802]]),
 '              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00      9243\n           1       0.95      0.98      0.96      6762\n           2       0.98      0.89      0.93       420\n           3       0.96      0.92      0.94      4135\n\n    accuracy                           0.97     20560\n   macro avg       0.97      0.95      0.96     20560\nweighted avg       0.97      0.97      0.97     20560\n')

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9243
           1       0.95      0.98      0.96      6762
           2       0.98      0.89      0.93       420
           3       0.96      0.92      0.94      4135

    accuracy                           0.97     20560
   macro avg       0.97      0.95      0.96     20560
weighted avg       0.97      0.97      0.97     20560

Το μοντέλο παρουσιάζει υψηλή ακρίβεια (97%) και ισορροπημένη απόδοση μεταξύ των κατηγοριών.

### Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest

# Δημιουργία και εκπαίδευση του μοντέλου Isolation Forest
iso_forest = IsolationForest(contamination=0.01, random_state=42)
iso_forest.fit(X_train)

# Πρόβλεψη ανωμαλιών στο σύνολο δοκιμής
y_pred_anomalies = iso_forest.predict(X_test)

# Αντικατάσταση των ετικετών με ανωμαλίες (-1) σε 1 και των κανονικών (1) σε 0
y_pred_anomalies = [1 if x == -1 else 0 for x in y_pred_anomalies]

# Εμφάνιση αποτελεσμάτων
print(pd.Series(y_pred_anomalies).value_counts())


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


0    20346
1      214
Name: count, dtype: int64
